# Продуктовая аналитика. Сириус 2025

Жернаков Иссайя Максимович

# Предварительный анализ

### Загрузка необходимых библиотек

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import chi2_contingency, mannwhitneyu, ks_2samp, f_oneway
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import GLSAR
import warnings
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
# Игнорирование предупреждений
warnings.simplefilter(action='ignore', category=FutureWarning)

### Загрузка датасета

In [ ]:
df = pd.read_csv("/Users/issaya/Desktop/сириус/Games Dataset.csv")
df

In [ ]:
df.info()
df.describe()

Исходя из информации о датасете, мы видим, что пропущенных значений нет. Приведем категориальные переменные к категориальному типу данных и снова выведем описательные статистики.

In [ ]:
(df['monthly_income_amt'] < 13890).sum()

In [ ]:
min_income = 13890

df = df[df['monthly_income_amt'] >= min_income]

df = df[df['age'] >= 14]

df['steam_popularity_score'] = df['steam_popularity_score'].replace(0, 5001)

df.reset_index(drop=True, inplace=True)

In [ ]:
cat_columns = ['category_id', 'gender_cd', 'education_level', 'city_nm', 'category_name']
df[cat_columns] = df[cat_columns].astype('category')


def iqr_bounds(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return lower, upper

cols = ['good_price', 'monthly_income_amt']
mask = np.ones(len(df), dtype=bool)

for col in cols:
    lower, upper = iqr_bounds(df[col])
    mask &= df[col].between(lower, upper)

df_clean = df[mask].copy().reset_index(drop=True)
df_clean.describe()

Согласно описательной статистике, количество товаров в каждом заказе всегда равно одному. Также мы видим отрицательные значения в зарплате (около 20% от общей выборки), скорее всего это ошибка в сборе данных, поэтому данным наблюдениям не стоит доверять. Значения зарплаты, которые оказываются ниже минимального размера оплаты труда на 2022 год, будут удалены из массива. 

Так как у нас датасет показывает список покупок игр нам следует рассматривать наблюдения, учитывая возраст покупателей. Наблюдения, в которых указан возраст ниже 14 лет (с данного возраста можно оформить банковскую карту, чтобы совершать покупки онлайн), будут расцениваться как ошибка покупателя при выборе своего возраста и поэтому они будут удалены из общей выборки.

Позиции игр в рейтинге топ-5000, равные нулю, интерпретируем как отсутствие игры в списке, и заменим на значение 5001, поскольку точное место таких игр неизвестно. После всех преобразований сбросим индексы для корректности.


In [ ]:
print("\nРазмер очищенного датасета:", df_clean.shape)

# №1. Исследование портретов покупателей

### 1. Как игроки отличаются от других клиентов Т-Банка?


Для того, чтобы сравнить игроков с остальными клиентами Т-банка необходимо найти данные, которые имеют схожие характеристики пользователей. Данный массив был найден в Базе данных DANO 2023-2024. Название '№6 Банк России: всероссийское обследование домохозяйств по потребительским финансам'. В датасете представлены данные по пятой волне (2022 год) Всероссийского обследования домохозяйств по потребительским финансам, опрос был проведен Банком России. Массив имеет достаточно много характеристик для сравнения с нашим исходным массивом. 
Однако, исходя из названия, в массиве выборка охватывает не только клиентов различных банков. Поэтому массив подвергся глубокой фильтрации по нескольким факторам:
1. Выявление клиентов банка.
После первичного анализа данных было выдвинуто предположение, что для клиентов банка должно выполнятся хотя бы одно условие:

а) человек должен оплачивать покупки безналичным способом

б) человек имеет кредит 

в) человек имеет вклад или сберегательный счет 

Если ни одно условие не выполняется, единица наблюдения удаляется из массива.

2. Фильтрация по населенным пунктам.
Первоначальный массив покупателей игр рассматривает только города с населением более одного миллиона. Поэтому для более точного сравнение из массива Центрального Банка были удалены наблюдения, которые были получены из маленьких по объему населения городов.

3. Выявление групп-городов для сравнения.

Так, как количество городов, в которых есть наблюдение отличается в двух датасетах, было принято решение разделить города на 3 группы: Москва, Санкт-Петербург, Регионы. Последняя группа включает в себя все остальные города.

4. Обработка пропущенных значений

Массив ЦБ имеет много пропущенных значений в колонке 'Зарплата'. Пропущенные значения были заменены медианной зарплатой по городу.

После вышеуказанных фильтраций остается проблема с выявлением клиентов Т-банка из общего массива клиентов банковских услуг. 

Будет сделано предположение, что клиенты Т-банка имеют незначительные различия между остальными клиентами банковского сектора и, следовательно, в данном проекте клиенты Т-банка будут приравнены к общей выборке всех клиентов банковских услуг.


#### Обработка датасета ЦБ

In [ ]:
cb = pd.read_excel('/Users/issaya/Desktop/сириус/central_bank_dataset.xlsx')
cb

Сохранение только необходимых столбцов для дальнейшего анализа (название всех столбцов и их характеристика находятся в файле ЦБ, который приложен к решению.

In [ ]:
cb = cb[['id_i', 'psu', 'gr_educ', 'sett_typ', 'size', 'i_h4', 'i_h5',
         'pp10_3', 'p10271', 'p93_1y', 'k69_n_s_r']]

In [ ]:
edu_map = {
    'Среднее специальное': 'SCH',
    'Среднее общее и ниже': 'SCH',
    'Высшее и неполное высшее': 'GRD',
    'НЕТ ОТВЕТА': np.nan,
    'ЗАТРУДНЯЮСЬ ОТВЕТИТЬ': np.nan,
    'ОТКАЗ ОТ ОТВЕТА': np.nan
}

gender_map = {'МУЖСКОЙ': 'M', 'ЖЕНСКИЙ': 'F'}
city_map = {'г.Москва': 'Москва', 'г.Санкт-Петербург': 'Санкт-Петербург'}
big_cities = ['свыше 1 млн.чел. (кроме столиц)', 'г.Москва', 'г.Санкт-Петербург']

cb = (
    cb.assign(
        gr_educ=lambda x: x['gr_educ'].replace(edu_map),
        i_h4=lambda x: x['i_h4'].replace(gender_map)
    )
    .loc[lambda x: x['size'].isin(big_cities)]
    .assign(
        city_nm=lambda x: x['size'].map(city_map).fillna('Регионы')
    )
    .copy()
)

In [ ]:
cb = cb.rename(columns={
    'id_i': 'client_id',
    'gr_educ': 'education_level',
    'i_h4': 'gender_cd',
    'i_h5': 'age',
    'pp10_3': 'cashless_type',
    'p10271': 'deposit_currency',
    'p93_1y': 'credit_start',
    'k69_n_s_r': 'monthly_income_amt'
})

Фильтрация по признакам использования банковских услуг

In [ ]:
cb = cb[~(
    (cb['cashless_type'] == 'Нет') &
    (cb['deposit_currency'].isna() | (cb['deposit_currency'] == 'НЕТ ОТВЕТА')) &
    (cb['credit_start'].isna() | cb['credit_start'].isin(['ЗАТРУДНЯЮСЬ ОТВЕТИТЬ', 'НЕТ ОТВЕТА']))
)]

cb['age'] = 2022 - cb['age']

cb['monthly_income_amt'] = cb.groupby('city_nm')['monthly_income_amt'] \
    .transform(lambda x: x.fillna(x.median()))

cb_clean = cb.reset_index(drop=True)

In [ ]:
cb_clean.head(5)

#### Сравнение покупателей игр с клиентами

In [ ]:
def analyze_groups(df_clean, cb_clean):
    df_1 = df_clean.copy()
    
    df_1['city_nm'] = df_1['city_nm'].apply(lambda x: x if x in ['Москва', 'Санкт-Петербург'] else 'Регионы')
    
    df_1['education_level'] = df_1['education_level'].replace('UGR', 'SCH')
    
    cb_clean['group'] = 'Клиенты Т-банка'
    df_1['group'] = 'Игроки'
    
    df_1 = df_1[df_1['age'] >= 18]
    
    def remove_outliers_iqr(df, column):
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        return df[(df[column] >= lower) & (df[column] <= upper)]
    
    cb_clean = remove_outliers_iqr(cb_clean, 'monthly_income_amt')
    df = pd.concat([cb_clean, df_1], ignore_index=True)
    
    gender_share = df.groupby(['group', 'gender_cd'])['client_id'].count().reset_index()
    gender_share['share'] = gender_share.groupby('group')['client_id'].transform(lambda x: x / x.sum())

    fig = px.bar(gender_share, x='gender_cd', y='share', color='group', barmode='group',
                 text=gender_share['share'].apply(lambda x: f'{x:.1%}'),
                 title='Сравнение пола: доли внутри группы')
    fig.update_layout(yaxis_tickformat='.0%')
    fig.show()

    print("Доли по полу:\n", gender_share.pivot(index='gender_cd', columns='group', values='share').round(3))
    chi2_gender, p_gender, _, _ = chi2_contingency(pd.crosstab(df['group'], df['gender_cd']))
    print(f'Пол (gender_cd): χ² = {chi2_gender:.2f}, p = {p_gender:.4f}')

    edu_share = df.groupby(['group', 'education_level'])['client_id'].count().reset_index()
    edu_share['share'] = edu_share.groupby('group')['client_id'].transform(lambda x: x / x.sum())

    fig = px.bar(edu_share, x='education_level', y='share', color='group', barmode='group',
                 text=edu_share['share'].apply(lambda x: f'{x:.1%}'),
                 title='Сравнение образования: доли внутри группы')
    fig.update_layout(yaxis_tickformat='.0%')
    fig.show()

    print("Доли по образованию:\n", edu_share.pivot(index='education_level', columns='group', values='share').round(3))
    chi2_edu, p_edu, _, _ = chi2_contingency(pd.crosstab(df['group'], df['education_level']))
    print(f'Образование: χ² = {chi2_edu:.2f}, p = {p_edu:.4f}')

    fig = px.box(df, x='group', y='monthly_income_amt', color='group', title='Сравнение зарплаты между группами')
    fig.show()
    print("Доход:\n", df.groupby('group')['monthly_income_amt'].describe().round(2))
    stat, p = mannwhitneyu(df[df['group'] == 'Клиенты Т-банка']['monthly_income_amt'],
                           df[df['group'] == 'Игроки']['monthly_income_amt'])
    print(f'Mann-Whitney U-test для дохода: статистика = {stat:.2f}, p = {p:.4f}')

    fig = px.box(df, x='group', y='age', color='group', title='Сравнение возраста между группами')
    fig.show()
    print("Возраст:\n", df.groupby('group')['age'].describe().round(2))
    stat_age, p_age = ks_2samp(df[df['group'] == 'Клиенты Т-банка']['age'],
                               df[df['group'] == 'Игроки']['age'])
    print(f'KS-тест по возрасту: статистика = {stat_age:.2f}, p = {p_age:.4f}')

    gender_city = df.groupby(['group', 'city_nm', 'gender_cd'])['client_id'].count().reset_index()
    gender_city['share'] = gender_city.groupby(['group', 'city_nm'])['client_id'].transform(lambda x: x / x.sum())

    fig = px.bar(gender_city, x='gender_cd', y='share', color='group',
                 facet_col='city_nm', barmode='group',
                 text=gender_city['share'].apply(lambda x: f'{x:.0%}'),
                 title='Пол по группам и городам')
    fig.update_layout(yaxis_tickformat='.0%', height=400)
    fig.show()

    edu_city = df.groupby(['group', 'city_nm', 'education_level'])['client_id'].count().reset_index()
    edu_city['share'] = edu_city.groupby(['group', 'city_nm'])['client_id'].transform(lambda x: x / x.sum())

    fig = px.bar(edu_city, x='education_level', y='share', color='group',
                 facet_col='city_nm', barmode='group',
                 text=edu_city['share'].apply(lambda x: f'{x:.0%}'),
                 title='Образование по группам и городам')
    fig.update_layout(yaxis_tickformat='.0%', height=450)
    fig.show()

    fig = px.box(df, x='group', y='monthly_income_amt', color='group',
                 facet_col='city_nm', title='Доход по группам и городам')
    fig.update_layout(height=450)
    fig.show()

    fig = px.box(df, x='group', y='age', color='group',
                 facet_col='city_nm', title='Возраст по группам и городам')
    fig.update_layout(height=450)
    fig.show()
analyze_groups(df_clean, cb_clean)

Графический и статистический анализ двух групп показал наличие сильных различий почти по всем характеристикам:
1. Пол

Клиенты Т-банка представляют примерно одинаковое соотношение между мужчинами и женщинами. Игроки наоборот зачастую мужчины. Тест Хи-квадрат показал статистическое различие.

2. Уровень образования

Даже учитывая, что массив ЦБ не обладает точной информацией об уровне образования по сравнению с датасетом с игроками, можно заметить, что больше половины клиентов Т-банка имеют только среднее образование. А игроки имеют ровно протиположную картину. Около 70% игроков имеют либо будут иметь в будущем высшее образование. Тест Хи-квадрат также выявил это различие.

3. Зарплата

Зарплата также сильно отличается. У игроков она на 34000 выше. Mann-Whitney U-test подтвердил различие.

4. Возраст

Игроки сильно моложе, чем остальные клиенты банка. Тест Колмогорова Смирнова подтвердил различие.

### 2. Чем отличаются заядлые покупатели игр?

Для выявления заядлых клиентов были рассчитаны два показателя активности: общее количество покупок и общая сумма покупок. Оба показателя были подвергнуты нормализации с целью приведения их к единой шкале, при этом предполагалось, что каждый из них оказывает равное влияние на итоговую оценку активности клиента.

На основе агрегированной метрики заядлости была выделена группа, составляющая 15% наиболее активных клиентов. Далее был проведён сравнительный анализ заядлых и незаядлых клиентов по следующим характеристикам: возраст, уровень дохода, интерес к играм, цена приобретаемых товаров, пол, уровень образования, город проживания и категория покупаемых игр. Анализ охватывал весь доступный временной интервал.

In [ ]:
def calculate_enthusiast_status(df):
    user_stats = (
        df.groupby('client_id')
        .agg(total_orders=('id', 'count'), total_spent=('good_price', 'sum'))
        .reset_index()
    )
    user_stats['norm_orders'] = (user_stats['total_orders'] - user_stats['total_orders'].min()) / (user_stats['total_orders'].max() - user_stats['total_orders'].min())
    user_stats['norm_spent'] = (user_stats['total_spent'] - user_stats['total_spent'].min()) / (user_stats['total_spent'].max() - user_stats['total_spent'].min())
    user_stats['enthusiast_score'] = 0.5 * user_stats['norm_orders'] + 0.5 * user_stats['norm_spent']
    
    threshold = user_stats['enthusiast_score'].quantile(0.85)
    user_stats['is_enthusiast'] = user_stats['enthusiast_score'] >= threshold
    
    df_labeled = df.merge(user_stats[['client_id', 'is_enthusiast']], on='client_id', how='left')
    df_labeled['Тип покупателя'] = df_labeled['is_enthusiast'].map({True: 'Заядлый', False: 'Незаядлый'})
    return df_labeled

def task_2(df_clean):
    df_labeled = calculate_enthusiast_status(df_clean)
    
    num_features = ['age', 'monthly_income_amt', 'steam_popularity_score', 'good_price']
    cat_features = ['gender_cd', 'education_level', 'city_nm', 'category_name']
    

    for feature in num_features:
        fig = px.box(df_labeled, x='Тип покупателя', y=feature, 
                     title=f'{feature}: распределение по типу покупателя',
                     color='Тип покупателя')
        fig.show()
        
        fig = px.histogram(df_labeled, x=feature, color='Тип покупателя', 
                           barmode='overlay', histnorm='probability density', nbins=40,
                           marginal='violin',
                           title=f'{feature}: плотность распределения по типу покупателя')
        fig.update_traces(opacity=0.6)
        fig.show()
    

    for feature in cat_features:
        grouped = (
            df_labeled
            .groupby([feature, 'Тип покупателя'])
            .size()
            .reset_index(name='count')
        )
        grouped['%'] = grouped.groupby(feature)['count'].transform(lambda x: 100 * x / x.sum())
        fig = px.bar(grouped, x=feature, y='%', color='Тип покупателя',
                     title=f'Распределение {feature} по типу покупателя',
                     text=grouped['%'].round(1).astype(str) + '%',
                     barmode='group')
        fig.update_layout(yaxis_title='Доля (%)', yaxis_tickformat='.0%')
        fig.show()
    

    print('\n Статистика по числовым признакам:')
    for feature in num_features:
        group_enth = df_labeled[df_labeled['is_enthusiast'] == True][feature].dropna()
        group_nonenth = df_labeled[df_labeled['is_enthusiast'] == False][feature].dropna()
        if len(group_enth) > 0 and len(group_nonenth) > 0:
            u_stat, u_p = mannwhitneyu(group_enth, group_nonenth, alternative='two-sided')
            ks_stat, ks_p = ks_2samp(group_enth, group_nonenth)
            
            print(f"\n{feature}:")
            print(f"  • Mann-Whitney U: статистика = {u_stat:.1f}, p = {u_p:.4f} → {'значимая разница ' if u_p < 0.05 else 'разницы нет '}")
            print(f"  • Kolmogorov-Smirnov: статистика = {ks_stat:.3f}, p = {ks_p:.4f} → {'разные распределения ' if ks_p < 0.05 else 'распределения схожи '}")
        else:
            print(f"{feature}: недостаточно данных для сравнения")
    

    print('\n Хи-квадрат для категориальных признаков:')
    for feature in cat_features:
        contingency = pd.crosstab(df_labeled[feature], df_labeled['is_enthusiast'])
        if contingency.shape[0] > 1 and contingency.shape[1] > 1:
            chi2, p, dof, _ = chi2_contingency(contingency)
            print(f"{feature}: χ² = {chi2:.2f}, p = {p:.4f}, df = {dof} → {'зависимость есть ' if p < 0.05 else 'различий нет '}")
        else:
            print(f"{feature}: недостаточно данных для сравнения")
    
    return df_labeled

In [ ]:
df_with_labels = task_2(df_clean)

Анализ графиков свидетельствует о незначительных различиях между заядлыми и незаядлыми клиентами по большинству характеристик. Вместе с тем заядлые клиенты демонстрируют более высокую активность в различных жанрах игр, что выражается в преимущественном количестве покупок.

Также среди заядлых клиентов отмечается более высокая доля лиц с высшим образованием, что видно на гистограмме распределения уровня образования (education_level). Существенных различий по половому признаку между группами не обнаружено.

Дополнительно установлено, что заядлые клиенты склонны выбирать игры с более высокой популярностью, что подтверждается более выраженной асимметрией и вытянутым распределением на boxplot популярности.

Проведённый статистический анализ подтвердил наличие значимых различий между заядлыми и незаядлыми клиентами. Для проверки различий применялись критерий Манна–Уитни и критерий Колмогорова–Смирнова. Выбор этих тестов обусловлен особенностями распределения данных: предпосылки нормальности не выполняются, что делает использование непараметрических методов наиболее корректным.

Критерий Манна–Уитни использовался для сравнения медианных значений между двумя независимыми выборками, тогда как критерий Колмогорова–Смирнова позволил оценить различия в формах распределений. Оба теста подтвердили наличие статистически значимых различий по ряду анализируемых характеристик.

### 3. Нацелены ли игры на разные аудитории?

Сначала проведем сегментацию пользователей, разбив их на группы по возрасту, доходу, образованию и гендеру. В этом задании мы рассматриваем весь временной отрезок. Затем строим распределение по категориям игр, рассчитав, сколько уникальных клиентов каждой группы покупали игры определённой категории и какая доля каждой группы приходится на каждую категорию, визуализируем это в виде тепловых карт (heatmap), чтобы увидеть, где есть перекосы.
Для каждой категории игр рассчитываемся усреднённые характеристики по потраченной сумме, цене одной игры, возрасту, доходу, популярности игры. Затем для этих соответствующих метрик выполняем анализ дисперсии ANOVA.


In [ ]:
def task_3(df_clean):
    df_clean['total_spent_per_order'] = df_clean['good_price']
    df_clean['avg_price'] = df_clean['good_price']

    df_clean['age_group'] = pd.cut(
        df_clean['age'],
        bins = [0, 17, 24, 34, 49, 100],
        labels = ['до 18', '18-24', '25-34', '35-49', '50+']
    )
    # Добавление новой колонки
    

    df_clean['income_group'] = pd.cut(
        df_clean['monthly_income_amt'],
        bins=[0, 30000, 60000, 90000, float('inf')],
        labels=['0-30k', '30k-60k', '60k-90k', '90k+']
    )

    def compute_dist(df, col):
        dist = df[['category_name', col, 'client_id']].drop_duplicates()
        dist = dist.groupby(['category_name', col], observed=False)['client_id'].nunique().reset_index(name='n_clients')
        total = dist.groupby('category_name')['n_clients'].transform('sum')
        dist['share'] = dist['n_clients'] / total
        return dist

    def plot_heatmap(dist_df, index_col, col_col, value_col, title, cmap="Blues"):
        heatmap_data = dist_df.pivot(index=index_col, columns=col_col, values=value_col)
        plt.figure(figsize=(12, 6))
        sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap=cmap, cbar_kws={'label': 'Доля'})
        plt.title(title)
        plt.ylabel(index_col)
        plt.xlabel(col_col)
        plt.xticks(rotation=0)
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()

    dists = {
        'gender_cd': 'Распределение гендеров по категориям игр',
        'age_group': 'Распределение возрастных групп по категориям игр',
        'income_group': 'Распределение доходных групп по категориям игр',
        'education_level': 'Распределение уровня образования по категориям игр'
    }

    metrics = ['total_spent_per_order', 'avg_price', 'steam_popularity_score', 'age', 'monthly_income_amt']
    desc_dict = {}
    all_results = {}

    if 'period_label' in df_clean.columns:
        print("\n Визуализация по периодам")
        for period in df_clean['period_label'].unique():
            df_period = df_clean[df_clean['period_label'] == period].copy()

            orders_count = df_period.groupby('client_id', observed=False)['order_day'].nunique().rename('unique_orders')
            agg = df_period.groupby(['client_id', 'category_name'], observed=False).agg({
                'total_spent_per_order': 'sum',
                'steam_popularity_score': 'mean',
                'age': 'first',
                'monthly_income_amt': 'first',
                'avg_price': 'mean'
            }).reset_index()
            agg = agg.merge(orders_count, on='client_id')

            desc = agg.groupby('category_name', observed=False).agg({
                'total_spent_per_order': 'mean',
                'avg_price': 'mean',
                'steam_popularity_score': 'mean',
                'age': 'mean',
                'monthly_income_amt': 'mean',
                'client_id': 'nunique'
            }).rename(columns={'client_id': 'n_clients'})
            desc_dict[period] = desc

            print(f"\n Период: {period}")
            for col, title in dists.items():
                dist = compute_dist(df_period, col)
                plot_heatmap(dist, 'category_name', col, 'share', f"{title} ({period})")

            period_results = {}
            for col in metrics:
                groups = df_period.groupby('category_name', observed=False)[col].apply(lambda x: x.dropna().values)
                if len(groups) < 2 or groups.apply(len).min() < 2:
                    period_results[col] = None
                else:
                    period_results[col] = f_oneway(*groups).pvalue

            sig = [k for k, p in period_results.items() if p is not None and p < 0.05]
            print(f"Значимые различия по {len(sig)} метрикам: {', '.join(sig) if sig else 'нет'}")
            all_results[period] = period_results
    else:
        print("\n Визуализация распределений по демографическим признакам (все данные)")
        for col, title in dists.items():
            dist = compute_dist(df_clean, col)
            plot_heatmap(dist, 'category_name', col, 'share', title)

        orders_count = df_clean.groupby('client_id', observed=False)['order_day'].nunique().rename('unique_orders')
        agg = df_clean.groupby(['client_id', 'category_name'], observed=False).agg({
            'total_spent_per_order': 'sum',
            'steam_popularity_score': 'mean',
            'age': 'first',
            'monthly_income_amt': 'first',
            'avg_price': 'mean'
        }).reset_index()
        agg = agg.merge(orders_count, on='client_id')

        desc = agg.groupby('category_name', observed=False).agg({
            'total_spent_per_order': 'mean',
            'avg_price': 'mean',
            'steam_popularity_score': 'mean',
            'age': 'mean',
            'monthly_income_amt': 'mean',
            'client_id': 'nunique'
        }).rename(columns={'client_id': 'n_clients'})
        desc_dict['all'] = desc

        results = {}
        for col in metrics:
            groups = df_clean.groupby('category_name', observed=False)[col].apply(lambda x: x.dropna().values)
            if len(groups) < 2 or groups.apply(len).min() < 2:
                results[col] = None
            else:
                results[col] = f_oneway(*groups).pvalue

        sig = [k for k, p in results.items() if p is not None and p < 0.05]
        print(f"Значимые различия по {len(sig)} метрикам: {', '.join(sig) if sig else 'нет'}")
        all_results['all'] = results

    return desc_dict, all_results


In [ ]:
def plot_stacked_bar_plotly(dist_df, index_col, col_col, value_col, title):
    fig = px.bar(
        dist_df,
        x=index_col,
        y=value_col,
        color=col_col,
        text=dist_df[value_col].apply(lambda x: f"{x:.1%}"),
        title=title,
        labels={index_col: "Категория игры", value_col: "Доля"},
        barmode="stack"
    )
    fig.update_layout(
        xaxis_title=None,
        yaxis_tickformat=".0%",
        legend_title=col_col,
        height=500,
        bargap=0.1
    )
    fig.update_traces(textposition='inside', textfont_size=12)
    fig.show()


def task_3(df_clean):
    df_clean['total_spent_per_order'] = df_clean['good_price']
    df_clean['avg_price'] = df_clean['good_price']

    df_clean['age_group'] = pd.cut(
        df_clean['age'],
        bins=[0, 17, 24, 34, 49, 100],
        labels=['до 18', '18-24', '25-34', '35-49', '50+']
    )

    df_clean['income_group'] = pd.cut(
        df_clean['monthly_income_amt'],
        bins=[0, 30000, 60000, 90000, float('inf')],
        labels=['0-30k', '30k-60k', '60k-90k', '90k+']
    )

    def compute_dist(df, col):
        dist = df[['category_name', col, 'client_id']].drop_duplicates()
        dist = dist.groupby(['category_name', col], observed=False)['client_id'].nunique().reset_index(name='n_clients')
        total = dist.groupby('category_name')['n_clients'].transform('sum')
        dist['share'] = dist['n_clients'] / total
        return dist

    dists = {
        'gender_cd': 'Гендерное распределение по категориям игр',
        'age_group': 'Возрастное распределение по категориям игр',
        'income_group': 'Доходное распределение по категориям игр',
        'education_level': 'Распределение образования по категориям игр'
    }

    metrics = ['total_spent_per_order', 'avg_price', 'steam_popularity_score', 'age', 'monthly_income_amt']
    desc_dict = {}
    all_results = {}

    if 'period_label' in df_clean.columns:
        print("\n Визуализация по периодам:")
        for period in df_clean['period_label'].unique():
            df_period = df_clean[df_clean['period_label'] == period].copy()

            orders_count = df_period.groupby('client_id', observed=False)['order_day'].nunique().rename('unique_orders')
            agg = df_period.groupby(['client_id', 'category_name'], observed=False).agg({
                'total_spent_per_order': 'sum',
                'steam_popularity_score': 'mean',
                'age': 'first',
                'monthly_income_amt': 'first',
                'avg_price': 'mean'
            }).reset_index()
            agg = agg.merge(orders_count, on='client_id')

            desc = agg.groupby('category_name', observed=False).agg({
                'total_spent_per_order': 'mean',
                'avg_price': 'mean',
                'steam_popularity_score': 'mean',
                'age': 'mean',
                'monthly_income_amt': 'mean',
                'client_id': 'nunique'
            }).rename(columns={'client_id': 'n_clients'})
            desc_dict[period] = desc
 
            print(f"\n Период: {period}")
            for col, title in dists.items():
                dist = compute_dist(df_period, col)
                plot_stacked_bar_plotly(dist, 'category_name', col, 'share', f"{title} ({period})")

            period_results = {}
            for col in metrics:
                groups = df_period.groupby('category_name', observed=False)[col].apply(lambda x: x.dropna().values)
                if len(groups) < 2 or groups.apply(len).min() < 2:
                    period_results[col] = None
                else:
                    period_results[col] = f_oneway(*groups).pvalue

            sig = [k for k, p in period_results.items() if p is not None and p < 0.05]
            print(f"Значимые различия по {len(sig)} метрикам: {', '.join(sig) if sig else 'нет'}")
            all_results[period] = period_results
    else:
        print("\n Визуализация по демографическим группам (все данные):")
        for col, title in dists.items():
            dist = compute_dist(df_clean, col)
            plot_stacked_bar_plotly(dist, 'category_name', col, 'share', title)

        orders_count = df_clean.groupby('client_id', observed=False)['order_day'].nunique().rename('unique_orders')
        agg = df_clean.groupby(['client_id', 'category_name'], observed=False).agg({
            'total_spent_per_order': 'sum',
            'steam_popularity_score': 'mean',
            'age': 'first',
            'monthly_income_amt': 'first',
            'avg_price': 'mean'
        }).reset_index()
        agg = agg.merge(orders_count, on='client_id')

        desc = agg.groupby('category_name', observed=False).agg({
            'total_spent_per_order': 'mean',
            'avg_price': 'mean',
            'steam_popularity_score': 'mean',
            'age': 'mean',
            'monthly_income_amt': 'mean',
            'client_id': 'nunique'
        }).rename(columns={'client_id': 'n_clients'})
        desc_dict['all'] = desc

        results = {}
        for col in metrics:
            groups = df_clean.groupby('category_name', observed=False)[col].apply(lambda x: x.dropna().values)
            if len(groups) < 2 or groups.apply(len).min() < 2:
                results[col] = None
            else:
                results[col] = f_oneway(*groups).pvalue

        sig = [k for k, p in results.items() if p is not None and p < 0.05]
        
        all_results['all'] = results

    return desc_dict, all_results

По тепловой карте заметим, что покупки во всех категориях игр совершают преимущественно мужчины. При этом в категориях предзаказов и карт оплаты все транзакции принадлежат исключительно мужчинам. Основная возрастная группа покупателей — это клиенты Т-банка от 18 до 29 лет: на них приходится наибольшее количество продаж практически во всех жанрах. Предзаказы чаще всего оформляют именно представители этой группы (75% случаев), а также пользователи в возрасте 40–49 лет (25%). Карты оплаты покупают либо клиенты до 18 лет, либо клиенты в возрасте от 30 до 39 лет — доли здесь распределены поровну. В категории «другое» покупки делятся между возрастами 18–29 и 30–39 лет (по 41%), а среди пользователей 40–49 лет помимо интереса к предзаказам также наблюдается склонность к покупкам в жанре онлайн (MMO). Подростки до 18 лет внутри своей возрастной группы чаще выбирают VR.

Интерес к картам оплаты и предзаказам проявляют пользователи с доходом от 60 до 90 тысяч рублей в месяц. Эта группа составляет половину всех предзаказов, в то время как остальные 50% равномерно делятся между доходами 30–60 тыс. и выше 90 тыс. рублей. Люди с доходом ниже 30 тыс. рублей предзаказов не оформляют. Среди них популярны в основном онлайн-игры (MMO). Группа с доходом 30–60 тыс. чаще выбирает VR, с доходом 60–90 тыс. — аркады, а среди тех, кто зарабатывает более 90 тыс., наиболее востребованной становится категория «другое», за ней следует подписка на Xbox.

По уровню образования интерес к картам оплаты и предзаказам характерен как для школьников, так и для пользователей с одним высшим образованием — эти две группы демонстрируют примерно одинаковую активность. Выпускники вузов чаще всего интересуются онлайн-играми (MMO), но также проявляют значительный интерес к подпискам на Xbox и играм для детей. Среди студентов самой популярной категорией стали новинки, а школьники чаще выбирают аниме, VR, гонки и детские игры. Новинки больше всего нравятся выпускникам, однако среди студентов это тоже наиболее востребованная категория.

In [ ]:
desc, results = task_3(df_clean)

В контексте дохода, анализ показал, что подписка на X-box более востребована среди лиц с высокими доходами. Это может быть связано с тем, что данная категория клиентов обладает большими финансовыми возможностями, что позволяет им приобретать не только подписки, но и другие дополнительные сервисы и продукты. Следовательно, для маркетинга и целевых предложений имеет смысл ориентироваться на эту группу потребителей, предлагая им премиум-пакеты или дополнительные опции.

Что касается возрастных характеристик, то основную долю среди покупателей игр для детей составляют лица в возрасте 25-34 лет. Это может говорить о том, что данная группа покупателей чаще всего является родителями, которые покупают игры для своих детей. Важно отметить, что эта возрастная категория активно использует цифровые платформы для развлечений, что подтверждается высоким интересом к таким продуктам, как игры для детей. Это открывает возможности для рекламных кампаний, направленных на родителей, предлагая им разнообразные игровые продукты, ориентированные на детей разных возрастов.

### 4. Справедливы ли выводы предыдущих пунктов на всем представленном временном промежутке?

Для более детального изучения динамики спроса проведён сравнительный анализ характеристик покупательского поведения в стандартные и аномальные периоды. Сравнение охватывает ключевые метрики, позволяя выявить отличия в структуре спроса и определить возможные факторы, повлиявшие на возникновение аномалий.

In [ ]:
df_clean['order_day'] = pd.to_datetime(df_clean['order_day'], errors='coerce')

df_clean['month'] = df_clean['order_day'].dt.to_period('M')

df_filtered = df_clean[df_clean['month'] <= pd.Period('2023-12', freq='M')]

monthly_order_count = df_filtered.groupby('month').size().reset_index(name='order_count')

monthly_order_count['month'] = monthly_order_count['month'].astype(str)

fig = px.line(monthly_order_count, x='month', y='order_count', 
              title='Количество заказов по месяцам', 
              labels={'month': 'Месяц', 'order_count': 'Количество заказов'},
              line_shape='linear')

fig.update_layout(
    xaxis_title="Месяц",
    yaxis_title="Количество заказов",
    xaxis=dict(tickmode='array', tickvals=monthly_order_count['month'], tickangle=45)
)

fig.show()


На графике зависимости спроса от месяца были выявлены пониженные значения в период с февраля 2022 года по октябрь 2023 года, а также выраженный всплеск спроса в декабре 2022 года. В данном разделе будет проведено сравнение распределений между основными периодами и периодами, характеризующимися аномальными значениями спроса.

In [ ]:
df_clean['month'] = pd.to_datetime(df_clean['order_day']).dt.to_period('M')

early_months_1 = pd.period_range('2022-02', '2022-03', freq='M')
main_months_1 = [m for m in df_clean['month'].unique() if m not in early_months_1]

def get_period_dfs(df, early_months, main_months):
    df_early = df[df['month'].isin(early_months)].copy()
    df_early['period_label'] = 'Фев-Март 2022'
    
    df_main = df[df['month'].isin(main_months)].copy()
    df_main['period_label'] = 'Апр 2022 — Окт 2023'
    
    return df_early, df_main

df_early_1, df_main_1 = get_period_dfs(df_clean, early_months_1, main_months_1)

print("Размер df_early_1:", df_early_1.shape)
print("Размер df_main_1:", df_main_1.shape)
print("Месяцы в df_early_1:", df_early_1['month'].unique())
print("Месяцы в df_main_1:", df_main_1['month'].unique())


def compare_distributions_smoothed(df1, df2, column, title, bins=50, smooth_window=3):
    hist1, bin_edges = np.histogram(df1[column], bins=bins, density=True)
    hist2, _ = np.histogram(df2[column], bins=bin_edges, density=True)
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    smooth_hist1 = pd.Series(hist1).rolling(smooth_window, center=True).mean()
    smooth_hist2 = pd.Series(hist2).rolling(smooth_window, center=True).mean()

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=smooth_hist1,
        mode='lines',
        name='Фев-Март 2022',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=smooth_hist2,
        mode='lines',
        name='Апр 2022 — Окт 2023',
        line=dict(color='orange')
    ))

    fig.update_layout(
        title=f'Сравнение распределения: {title}',
        xaxis_title=title,
        yaxis_title='Плотность',
        template='plotly_white'
    )

    fig.show()

numeric_columns = ['good_price', 'monthly_income_amt', 'age', 'steam_popularity_score']

for col in numeric_columns:
    print(f"\n=== Анализ распределения для {col} ===")
    compare_distributions_smoothed(df_early_1, df_main_1, col, col)

In [ ]:
df_clean['month'] = pd.to_datetime(df_clean['order_day']).dt.to_period('M')

december_months = pd.period_range('2022-12', '2022-12', freq='M')
other_months = [m for m in df_clean['month'].unique() if m not in december_months]


def get_december_vs_rest(df, dec_months, other_months):
    df_dec = df[df['month'].isin(dec_months)].copy()
    df_dec['period_label'] = 'Декабрь 2022'

    df_rest = df[df['month'].isin(other_months)].copy()
    df_rest['period_label'] = 'Остальной период'

    return df_dec, df_rest


df_december, df_rest = get_december_vs_rest(df_clean, december_months, other_months)

print("Размер df_december:", df_december.shape)
print("Размер df_rest:", df_rest.shape)
print("Месяцы в df_december:", df_december['month'].unique())
print("Месяцы в df_rest:", df_rest['month'].unique())


def compare_distributions_smoothed(df1, df2, column, title, bins=50, smooth_window=3):
    hist1, bin_edges = np.histogram(df1[column], bins=bins, density=True)
    hist2, _ = np.histogram(df2[column], bins=bin_edges, density=True)
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    smooth_hist1 = pd.Series(hist1).rolling(smooth_window, center=True).mean()
    smooth_hist2 = pd.Series(hist2).rolling(smooth_window, center=True).mean()

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=smooth_hist1,
        mode='lines',
        name=df1['period_label'].iloc[0],
        line=dict(color='green')
    ))

    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=smooth_hist2,
        mode='lines',
        name=df2['period_label'].iloc[0],
        line=dict(color='gray')
    ))

    fig.update_layout(
        title=f'Сравнение распределения: {title}',
        xaxis_title=title,
        yaxis_title='Плотность',
        template='plotly_white'
    )

    fig.show()


numeric_columns = ['good_price', 'monthly_income_amt', 'age', 'steam_popularity_score']

for col in numeric_columns:
    print(f"\n=== Сравнение Декабря 2022 и остального периода для {col} ===")
    compare_distributions_smoothed(df_december, df_rest, col, col)

Распределения таких характеристик, как цена, доход, популярность на Steam и возраст клиентов, между различными периодами демонстрируют схожесть. Линии плотности распределения на графиках проходят вдоль друг друга, что свидетельствует о наличии общих закономерностей в этих метриках для разных временных интервалов. Таким образом, несмотря на аномальные изменения в некоторых месяцах, распределения в целом сохраняют свою стабильность.

Этот факт позволяет заключить, что поведение клиентов и их предпочтения в отношении цены, дохода, популярности игр на Steam и возраста остаются относительно стабильными как в стандартные, так и в аномальные периоды. Таким образом, результаты анализа, сделанные для одного периода, можно с уверенностью экстраполировать на весь временной промежуток.

# №2. Эластичность и спрос

## Предварительная обработка массива

In [ ]:
elasticity = df_clean[['category_name', 'order_day', 'good_price', 'good_cnt', 'gender_cd', 
                       'age', 'monthly_income_amt', 'steam_popularity_score', 'education_level']].copy()

elasticity['order_day'] = pd.to_datetime(elasticity['order_day'], errors='coerce')
elasticity['order_month'] = elasticity['order_day'].dt.month

edu_map = {
    'SCH': 1,  # начальное/среднее
    'UGR': 2,  # неполное высшее
    'GRD': 2,  # высшее
    'PGR': 2,  # два высших
    'ACD': 2  # ученая степень
}
elasticity['education_level'] = elasticity['education_level'].map(edu_map).astype(float)

elasticity = elasticity.groupby(['category_name', 'good_price']).agg(
    good_cnt=('good_cnt', 'sum'),
    male_share=('gender_cd', lambda x: (x == 'M').mean()),
    avg_age=('age', 'mean'),
    median_income=('monthly_income_amt', 'median'),
    median_popularity=('steam_popularity_score', 'median'),
    month_mode=('order_month', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
    edu_level=('education_level', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
).reset_index()

elasticity.rename(columns={
    'male_share': 'male_share',
    'avg_age': 'avg_age',
    'median_income': 'median_income',
    'median_popularity': 'median_popularity',
    'month_mode': 'month_mode'
}, inplace=True)

elasticity.dropna(inplace=True)

elasticity.sort_values(by=['category_name', 'good_price'], ascending=[True, False], inplace=True)

elasticity['cum_cnt'] = elasticity.groupby('category_name')['good_cnt'].cumsum()

elasticity['price_to_income'] = elasticity['good_price'] / elasticity['median_income']
scaler = StandardScaler()
elasticity['price_to_income'] = scaler.fit_transform(elasticity[['price_to_income']])

elasticity['edu_level_sq'] = elasticity['edu_level'] ** 2

elasticity['log_price'] = np.log1p(elasticity['good_price'])
elasticity['log_price_to_income'] = np.log1p(elasticity['price_to_income'])
elasticity['log_price_squared'] = elasticity['log_price'] ** 2
elasticity['log_price_x_popularity'] = elasticity['log_price'] * elasticity['median_popularity']
elasticity['log_price_sq_x_popularity'] = elasticity['log_price_squared'] * elasticity['median_popularity']
elasticity['log_popularity'] = np.log1p(elasticity['median_popularity'])
elasticity['log_cnt'] = np.log1p(elasticity['good_cnt'])
elasticity['log_cum_cnt'] = np.log1p(elasticity['cum_cnt'])

elasticity['month_mode'] = (elasticity['month_mode'] == 12).astype(int)

def map_genre(row):
    if row in ['Предзаказы', 'Карты оплаты']:
        return None
    elif row == 'Скидки':
        return 'Скидки'
    elif row in ['Экшн', 'Шутеры', 'Файтинги']:
        return 'Экшн'
    elif row in ['Ролевые (RPG)', 'Симуляторы', 'Стратегии']:
        return 'Ролевые игры и стратегии'
    elif row in ['Приключения', 'Хоррор']:
        return 'Приключения и Хоррор'
    elif row in ['Инди', 'Казуальные игры']:
        return 'Инди и казуальные игры'
    else:
        return 'Другие'

elasticity['category_group'] = elasticity['category_name'].apply(map_genre)

group_dummies = pd.get_dummies(elasticity['category_group'], prefix='genre', drop_first=True)

elasticity = pd.concat([elasticity, group_dummies], axis=1)


In [ ]:
elasticity.columns

In [ ]:
elasticity

## 1. Исследование текущих схем выставления цены и их эффективности

### Текущая ценовая политика

In [ ]:
fig = px.histogram(elasticity, x='good_price', nbins=50, title='Распределение цен на игры',
                   labels={'good_price': 'Цена'}, opacity=0.75)

fig.update_layout(
    title='Цена vs Спрос',
    xaxis_title='Цена',
    yaxis_title='Спрос',
    template='plotly_white',
    bargap=0.1
)

fig.show()

Типичный график, который объясняет модель рынка. Множество игр имеют низкие цены (до 20 или 100), что в общем-то соответствует стандартной практике для игр среднего и низкого уровня. Некоторые игры (например, премиум-класса или особо успешные) имеют высокие цены, но таких игр гораздо меньше, и их количество резко снижается с ростом цены.

In [ ]:
fig.add_trace(go.Scatter(
    x=elasticity['good_price'], y=elasticity['cum_cnt'],
    mode='markers', name='Продажи',
    marker=dict(color='blue', size=8, opacity=0.6)
))

fig.update_layout(
    title='Связь между ценой и спросом',
    xaxis_title='Цена',
    yaxis_title='Спрос',
    template='plotly_white'
)

fig.show()

График имеет вполне обычный график спроса (при увеличении цены, снижаются продажи). Однако график имеет свою особенность. На нем можно увидеть несколько линий, которые имеют разный угол наклона. Следует понять с чем связано такое поведение графика и какие характеристики он скрывает. Попробуем подсветить наблюдения жанром (каждый цвет - определенный жанр).

In [ ]:
fig = go.Figure()

for category in elasticity['category_name'].unique():
    category_data = elasticity[elasticity['category_name'] == category]
    fig.add_trace(go.Scatter(
        x=category_data['good_price'], y=category_data['cum_cnt'],
        mode='markers',
        name=category,
        marker=dict(size=8, opacity=0.6),
        text=category_data['category_name']
    ))

fig.update_layout(
    title='Цена vs Продажи по категориям жанров',
    xaxis_title='Цена',
    yaxis_title='Продажи',
    template='plotly_white',
    legend_title="Жанры"
)

fig.show()

Теперь стало понятнее. Каждая линия - определенный жанр. Разные жанры имеют свою функцю спроса, что вполне логично. Для дачи рекомендаций можно разделить жанры на 2 группы.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=elasticity['good_price'], 
    y=elasticity['cum_cnt'],
    mode='markers',
    marker=dict(
        size=8,
        color=elasticity['median_popularity'],  # Цвет в зависимости от популярности
        colorscale='Viridis',  # Градация по цвету
        colorbar=dict(title='Популярность')
    ),
    text=elasticity['category_name'],
    name='Цена vs Продажи'
))

fig.update_layout(
    title='Цена vs Спрос с градацией по популярности',
    xaxis_title='Цена',
    yaxis_title='Спрос',
    template='plotly_white',
    legend_title="Жанры"
)

fig.show()

Теперь давайте подсветим наблюдения в зависимости от популярности игры. Мы видим, что игры, не входящие в топ-5000 примерно равномерно распределены на всей линии спроса (есть небольшое скопление на левом краю). Наблюдений с низким рейтингом настолько много, что они не дают четко определить распределение других игр. Уберем их на графике ниже.

In [ ]:
filt = elasticity[elasticity['median_popularity'] <= 5000]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=filt['good_price'],
    y=filt['cum_cnt'],
    mode='markers',
    marker=dict(
        size=8,
        color=filt['median_popularity'],  # Цвет в зависимости от популярности
        colorscale='Viridis',  # Градация по цвету
        colorbar=dict(title='Популярность')
    ),
    text=filt['category_name'],
    name='Цена vs Продажи'
))

fig.update_layout(
    title='Цена vs Спрос (только популярность ≤ 5000)',
    xaxis_title='Цена',
    yaxis_title='Спрос',
    template='plotly_white',
    legend_title="Жанры"
)

fig.show()

На графике мы видим интересную зависимость. В каждом жанре наблюдается схожее распределение продаж в зависимости от цены.
Чем ниже рейтинг игры, тем больше объем продаж и ниже ее цена. Чем выше рейтинг игры, тем ниже ее совокупный спрос и выше цена. Это можно объяснить следующим образом:
1. Игры с низкой ценой становятся привлекательными для широкой аудитории. Разработчики игр могут это для привлечения внимания к большому числу покупателей, которые не так сильно заинтересованы в высококачественном контенте.

2. Дорогие, но малопопулярные игры могут иметь высокую цену, так как они ориентированы на более узкую и платежеспособную аудиторию.

3. Нишевые игры (с низким рейтингом) могут быть более доступными по цене, привлекая пользователей, которые заинтересованы в том, чтобы попробовать что-то новое, но не хотят рисковать большим бюджетом.
4. Популярные игры (с высоким рейтингом), которые допустим имеют несколько частей, могут быть достаточно дорогими. Обычно такие игры имеют свою фан-базу, которая готова платить за хороший продукт.



### Оценка эффективности ценовой политики

Анализ текущей ценовой политики показал целесообразность рассматривания жанров индивидуально, так как каждый жанр имеет разную динамику спроса.

Будем действовать по плану ниже:
1. Оценка кол-ва данных с помощью графика выручка/цена
2. Подсчет оптимальной цены
3. Оценка эффективности

Оценка кол-ва данных с помощью графика выручка/цена

In [ ]:
elasticity['revenue'] = elasticity['good_price'] * elasticity['cum_cnt']
fig = px.line(elasticity,
              x='good_price', y='revenue',
              facet_col='category_name',
              facet_col_wrap=4,
              title="Зависимость выручки от цены по жанрам",
              labels={'good_price': 'Цена', 'revenue': 'Выручка'},
              height=800)
fig.update_layout(showlegend=False)
fig.show()

После построение графика выручка/цена мы видим, что некоторым жанрам не хватает данных, чтобы точно определить точку максимальной выручки. Удалим эти жанры при подсчете оптимальной выручки при помощи максимизации выручки.

Подсчет оптимальной цены

In [ ]:
results = []

for genre in elasticity['category_name'].unique():
    df_genre = elasticity[elasticity['category_name'] == genre]

    if len(df_genre) < 10:
        continue  # Пропускаем малые группы

    X = sm.add_constant(df_genre['good_price'])
    y = df_genre['cum_cnt']
    model = sm.OLS(y, X).fit()

    a = model.params['const']
    b = model.params['good_price']

    if b >= 0:
        optimal_price = None
    else:
        optimal_price = -a / (2 * b)

    current_price = df_genre['good_price'].mean()

    current_cnt = a + b * current_price
    optimal_cnt = a + b * optimal_price if optimal_price else None
    current_revenue = current_price * current_cnt
    optimal_revenue = optimal_price * optimal_cnt if optimal_price else None

    efficiency = current_revenue / optimal_revenue if optimal_revenue else None

    results.append({
        'category_name': genre,
        'a_coeff': a,
        'b_coeff': b,
        'current_price': current_price,
        'optimal_price': optimal_price,
        'current_revenue': current_revenue,
        'optimal_revenue': optimal_revenue,
        'efficiency_ratio': efficiency,
        'R_squared': model.rsquared
    })

revenue_df = pd.DataFrame(results)

revenue_df_sorted = revenue_df.sort_values(by='efficiency_ratio', ascending=False)

revenue_df_sorted[
    ['category_name', 'current_price', 'current_price']]

Оценка эффективности

In [ ]:
# Отфильтруем жанры с эффективностью < 90% и текущей ценой ниже оптимальной
underpriced = revenue_df[
    (revenue_df['efficiency_ratio'] < 0.9) & 
    (revenue_df['optimal_price'].notnull()) & 
    (revenue_df['current_price'] < revenue_df['optimal_price'])
]

print(" Жанры с эффективностью < 90% и заниженной ценой:")
underpriced[['category_name', 'current_price', 'optimal_price']]

Разработчикам жанров 'Гонки', 'Инди', 'Для детей', 'Спорт' следует пересмотреть свою ценовую политику, так как они нынешняя средняя цена гораздо ниже, чем оптимальная.


## 2. Ограничения построения эластичности и предварительная модель

Точно построить модель эластичности для каждой игры не представляется возможным из-за маленького объема наблюдений. В связи с этим спрос будет предсказан на определнный жанр. Это позволит увеличить выборку и повысить точность. Далее будет проведен анализ жанров и количество наблюдений в каждом. 

In [ ]:
summary = elasticity.groupby('category_name').agg(
    count=('good_price', 'count'),
    price_std=('good_price', 'std'),
    price_min=('good_price', 'min'),
    price_max=('good_price', 'max')
).reset_index()

problematic = summary[(summary['count'] < 50) | (summary['price_std'] < 0.1)]

print(" Категории, где сложно моделировать эластичность:")
problematic

In [ ]:
elasticities = []

for genre in elasticity['category_name'].unique():
    df_genre = elasticity[elasticity['category_name'] == genre]
    
    if len(df_genre) < 80:
        continue

    X = sm.add_constant(df_genre['log_price'])
    y = df_genre['log_cum_cnt']

    model = sm.OLS(y, X).fit()

    elast = model.params['log_price']
    elasticities.append({
        'category_name': genre,
        'elasticity': elast,
        'R_squared': model.rsquared
    })

elasticity_df = pd.DataFrame(elasticities)
elasticity_df

Анализ ценовой эластичности показал следующие результаты:

	•	Наибольшую ценовую эластичность (наибольшую чувствительность спроса к изменению цены) продемонстрировали жанры: Бестселлеры (-2.43), Anime (-1.26), Файтинги (-1.12)
Это значит, что снижение цены на игры в этих жанрах приводит к заметному росту спроса, а повышение цены — к его сильному снижению.

	•	Наименьшая ценовая эластичность (спрос реагирует слабее) наблюдается у жанров: Экшн (-0.66), Ролевые (RPG) (-0.73),Инди (-0.76)
Игры в этих жанрах менее чувствительны к изменению цены, что создаёт больше возможностей для ценовой оптимизации без резкого падения спроса.

	•	У всех жанров эластичность оказалась отрицательной, что является ожидаемым результатом: повышение цены на игры приводит к снижению спроса.

## 3. Выявление факторов, которые влияют на спрос

In [ ]:
gender_demand = df_clean.groupby('gender_cd')['good_cnt'].sum().reset_index()
fig = px.bar(gender_demand, x='gender_cd', y='good_cnt',
             title='Средний спрос по полу', labels={'cum_cnt': 'Средний спрос'})
fig.show()
print(gender_demand.round(2))

df_clean['age_group'] = pd.cut(df_clean['age'], bins=[0, 18, 25, 35, 45, 60, 100],
                               labels=['<18', '18–25', '26–35', '36–45', '46–60', '60+'])

age_demand = df_clean.groupby('age_group')['good_cnt'].sum().reset_index()
fig = px.bar(age_demand, x='age_group', y='good_cnt',
             title='Средний спрос по возрастным группам')
fig.show()
print(age_demand.round(2))

edu_demand = df_clean.groupby('education_level')['good_cnt'].sum().reset_index()
fig = px.bar(edu_demand, x='education_level', y='good_cnt',
             title='Средний спрос по уровню образования')
fig.show()
print(edu_demand.round(2))

df_clean['income_group'] = pd.qcut(df_clean['monthly_income_amt'], q=4,
                                   labels=['Низкий доход', 'Средний-', 'Средний+', 'Высокий доход'])

income_demand = df_clean.groupby('income_group')['good_cnt'].sum().reset_index()
fig = px.bar(income_demand, x='income_group', y='good_cnt',
             title='Средний спрос по уровню дохода')
fig.show()
print(income_demand.round(2))

def popularity_group(score):
    if score <= 1000:
        return '1–1000'
    elif score <= 2000:
        return '1000–2000'
    elif score <= 3000:
        return '2000–3000'
    elif score <= 4000:
        return '3000–4000'
    elif score < 5000:
        return '4000–5000'
    else:
        return '5000+'

df_clean['popularity_group'] = df_clean['steam_popularity_score'].apply(popularity_group)
popularity_grouped = df_clean.groupby('popularity_group')['good_cnt'].sum().reset_index()
popularity_order = {'1–1000': 1, '1000–2000': 2, '2000–3000': 3,
                    '3000–4000': 4, '4000–5000': 5, '5000+': 6}
popularity_grouped['popularity_numeric'] = popularity_grouped['popularity_group'].map(popularity_order)
print("Корреляция между популярностью в Steam и спросом:\n")
print(popularity_grouped[['popularity_numeric', 'good_cnt']].corr())

fig = px.bar(popularity_grouped.sort_values('popularity_numeric'),
             x='popularity_group', y='good_cnt',
             title='Суммарные покупки по уровням популярности в Steam',
             labels={'popularity_group': 'Популярность (Steam)', 'good_cnt': 'Общее количество покупок'})
fig.show()

df_clean['month'] = df_clean['month'].dt.to_timestamp()
month_demand = df_clean.groupby('month')['good_cnt'].sum().reset_index()
fig = px.line(month_demand, x='month', y='good_cnt',
              title='Средний спрос по месяцам')
fig.show()
print(month_demand.round(2))

    1.	Пол оказывает влияние на объем продаж, что подтверждается различиями в покупательской активности между мужчинами и женщинами.
	2.	Возраст клиентов демонстрирует неравномерное распределение, что позволяет четко выделить целевую аудиторию,ориентированную на определённые возрастные группы.
	3.	Уровень образования влияет на спрос: с увеличением степени образования наблюдается рост покупательской активности. Однако следует отметить, что покупатели с неоконченным высшим образованием совершают меньшие покупки по сравнению с теми, кто завершил высшее образование. Это, вероятно, связано с тем, что выпускники вузов обладают большей финансовой независимостью, что способствует увеличению объема их покупок.
	4.	Доход имеет минимальное влияние на спрос, что указывает на возможное отсутствие явной зависимости между уровнем дохода и покупательской активностью в рассматриваемой выборке.
	5.	Популярность на Steam демонстрирует обратную зависимость: чем ниже популярность игры, тем выше спрос на неё. Это может свидетельствовать о предпочтении аудитории к менее известным, но потенциально более интересным играм.
	6.	Дата покупок также может иметь влияние на спрос, что указывает на возможную сезонность. Наблюдаются резкие скачки и просадки в определённые периоды, что может быть связано с маркетинговыми акциями, праздничными распродажами или другими внешними факторами.

## 4. Регрессионная модель

После анализа факторов, оказывающих влияние на спрос, было принято решение добавить в регрессионную модель такие переменные, как цена, образование, популярность на Steam, жанр игры и возраст. Уровень дохода не оказывает существенного влияния на спрос, что подтверждается соответствующим графиком, на котором не наблюдается ярко выраженной зависимости.

Кроме того, после анализа влияния факторов на спрос стало понятно, необходимо объединить степень образования в две категории: среднее (включая начальное и среднее образование) и высшее (включая неполное высшее, два высших образования и ученую степень). Это упрощение позволит лучше отразить влияние образовательного уровня на покупательскую активность.

In [ ]:
elasticity = elasticity.loc[:, ~elasticity.columns.duplicated()]

elasticity['education_level'] = elasticity['edu_level'].apply(lambda x: 1 if x == '2' else 0)

features = [
    'log_price', 'avg_age', 'edu_level',
    'genre_Инди и казуальные игры',
    'genre_Приключения и Хоррор',
    'genre_Ролевые игры и стратегии',
    'genre_Скидки',
    'genre_Экшн'
]

X = elasticity[features]
y = elasticity['cum_cnt']

scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=features)

X_scaled = sm.add_constant(X_scaled)

X_scaled = X_scaled.reset_index(drop=True)
y = y.reset_index(drop=True)

model_glsar = GLSAR(y, X_scaled, rho=1)
model_glsar.iterative_fit(10)
res = model_glsar.fit()

print(res.summary())

In [ ]:
price_change_pct = 0.20

expected_change = -3330.32 * (math.exp(price_change_pct) - 1)
print(f"При росте цены на 20%, ожидаемое снижение спроса: {expected_change:.0f} ед.")

In [ ]:
import plotly.graph_objects as go

coef_dict = {
    'Цена (лог)': abs(-3330.33),
    'Возраст': abs(-33.35),
    'Образование': abs(8.21),
    'Инди и казуальные игры': 555.47,
    'Приключения и Хоррор': 2161.76,
    'Ролевые игры и стратегии': 2193.08,
    'Скидки': 699.45,
    'Экшн': 488.29
}

fig = go.Figure(go.Bar(
    x=list(coef_dict.values()),
    y=list(coef_dict.keys()),
    orientation='h',
    marker_color='cornflowerblue'
))

fig.update_layout(
    title='Влияние факторов на кумулятивный спрос',
    xaxis_title='Абсолютное значение коэффициента (ед.)',
    yaxis_title='Фактор',
    template='plotly_white',
    height=500
)

fig.show()

# Заключение

Проведённый анализ позволил выявить ключевые особенности поведения покупателей игр и провести сравнительную характеристику различных групп клиентов.

### 1. Сравнение игроков с клиентами Т-банка
Графический и статистический анализ показал существенные различия между игроками и клиентами Т-банка почти по всем характеристикам.  
Игроки чаще являются мужчинами, моложе по возрасту, имеют более высокий уровень образования и доход по сравнению с общей клиентской базой банка.  
Эти различия были подтверждены с использованием тестов Хи-квадрат, Манна–Уитни и Колмогорова–Смирнова.

### 2. Характеристики заядлых покупателей игр
Анализ показал, что заядлые покупатели демонстрируют более высокую активность в различных жанрах игр, а также имеют более высокий уровень образования по сравнению с незаядлыми клиентами.  
Существенных различий по полу между группами не выявлено. Также отмечено, что заядлые клиенты предпочитают игры с большей популярностью на платформе Steam.  
Статистические тесты подтвердили наличие значимых различий между группами.

### 3. Нацеленность игр на разные аудитории
Игры и подписки на сервисы вроде X-box ориентированы на разные сегменты клиентов.  
Подписка на X-box более востребована среди лиц с высоким доходом, что открывает возможности для премиального позиционирования продуктов.  
Игры для детей чаще приобретаются клиентами в возрасте 25–34 лет, вероятно, родителями, что должно учитываться при разработке маркетинговых стратегий.

### 4. Стабильность выводов на протяжении времени
Распределения таких характеристик, как цена, доход, популярность на Steam и возраст, оставались стабильными в разные временные периоды.  
Это позволяет с уверенностью экстраполировать сделанные выводы на весь рассматриваемый временной промежуток.

### 5. Зависимость спроса от рейтинга игр
Игры с более низким рейтингом продаются лучше и имеют более низкую цену, тогда как высокорейтинговые игры характеризуются меньшим совокупным спросом и более высокой ценой.  
Это связано с разной стратегией позиционирования игр: дешёвые игры привлекают массовую аудиторию, тогда как дорогие рассчитаны на узкий круг более платежеспособных клиентов.

### 6. Эффективность ценовой политики
Рекомендовано пересмотреть ценовую политику для жанров «Гонки», «Инди», «Для детей», «Спорт», поскольку их текущая средняя цена заметно ниже оптимальной, что ограничивает возможности максимизации выручки.

### 7. Влияние факторов на спрос
- Пол, возраст, уровень образования, популярность на Steam и сезонность оказывают влияние на спрос.
- Уровень дохода продемонстрировал минимальное влияние на покупательскую активность в анализируемой выборке.

---

## Регрессионная модель

Рассмотрим влияние каждого из факторов в модели GLSAR, исходя из значений коэффициентов и их интерпретации.

### 1. Константа: 4729.23
Константа представляет собой базовое значение переменной **cum_cnt** (общее количество покупок), когда все остальные независимые переменные равны нулю.  
Когда все факторы равны нулю, ожидаемое количество покупок составляет 4729.23.

### 2. Логарифм цены: -3330.33
Коэффициент равен -3330.33, что указывает на инверсионную зависимость между ценой и количеством покупок:  
с увеличением цены на 1% количество покупок снижается в среднем на 3330.33 единиц.

### 3. Средний возраст: -33.35
С увеличением среднего возраста клиентов на 1 год количество покупок уменьшается на 33.35 единицы.  
Более возрастные клиенты совершают меньше покупок.

### 4. Уровень образования: 8.21
С повышением уровня образования клиента количество покупок увеличивается в среднем на 8.21 единицу.

### 5. Жанры:
- **Инди и казуальные игры** (`genre_Инди и казуальные игры`): +555.47
- **Приключения и Хоррор** (`genre_Приключения и Хоррор`): +2161.76
- **Ролевые игры и стратегии** (`genre_Ролевые игры и стратегии`): +2193.08
- **Скидки** (`genre_Скидки`): +699.45
- **Экшн** (`genre_Экшн`): +488.29

Предпочтение данным жанрам приводит к увеличению количества покупок на указанные значения.

---

### Итог регрессионной модели

- **Цена** и **возраст** оказывают отрицательное влияние на количество покупок: с ростом цены и возраста покупок становится меньше.
- **Уровень образования** и выбор определённых **жанров** оказывают положительное влияние на количество покупок.
- Жанры **Приключения и Хоррор** и **Ролевые игры и стратегии** демонстрируют наибольшее влияние, что следует учитывать при формировании маркетинговых и продуктовых стратегий.